In [7]:
import dgl
import numpy as np
import torch

In [8]:
g = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]), num_nodes=6)
# Equivalently, PyTorch LongTensors also work.
g = dgl.graph((torch.LongTensor([0, 0, 0, 0, 0]), torch.LongTensor([1, 2, 3, 4, 5])), num_nodes=6)

# You can omit the number of nodes argument if you can tell the number of nodes from the edge list alone.
g = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]))

# Print the source and destination nodes of every edge.
print(g.edges())

(tensor([0, 0, 0, 0, 0]), tensor([1, 2, 3, 4, 5]))


In [9]:
g.ndata['x'] = torch.randn((6, 2))

## Drawing a graph with PyVis

In [10]:
import networkx as nx
import matplotlib.pyplot as plt

# G = nx.complete_graph(5)
G = nx.Graph()

elist = [('a', 'b', 5.0), ('b', 'c', 3.0), ('a', 'c', 1.0), ('c', 'd', 7.3)]
G.add_weighted_edges_from(elist)
G['a']['b']['color'] = 'red'

options = {
    'node_color': 'red',
    # 'node_size': 50,
    'width': 1,
    'with_labels':True, 
    'font_weight':'bold'
}

print(G['a']['b'])
# nx.draw(G, **options)
# nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)        ## Dar edge labels


{'weight': 5.0, 'color': 'red'}


In [11]:
dataset = dgl.data.CoraGraphDataset()
g = dataset[0]
options = {
    'node_color': 'black',
    'node_size': 20,
    'width': 1,
}

print(g.num_nodes)

# G = dgl.to_networkx(g)
# plt.figure(figsize=[15,7])
# nx.draw(G, **options)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
<bound method DGLHeteroGraph.num_nodes of Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={'__orig__': Scheme(shape=(), dtype=torch.int64)})>


In [12]:
## Easy example

from pyvis.network import Network
import networkx as nx

g = Network(height=500, width=500, notebook=True)
# g.toggle_hide_edge_on_drag(True)
g.barnes_hut()
g.from_nx(nx.davis_southern_women_graph())
g.show_buttons(filter_=True)
g.show('ex.html')

In [13]:
## Not yet worked-out example

import dgl
from pyvis.network import Network
import networkx as nx

dataset = dgl.data.CoraGraphDataset()

g = Network(height=800, width=800, notebook=True)
# g.barnes_hut()

netxG = nx.Graph(dataset[0].to_networkx())
print(netxG)

# g.from_nx(netxG)
# nt.show_buttons()
# g.show('ex.html')

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Graph with 2708 nodes and 5278 edges


In [14]:
## How to keep the label on edges ?

from pyvis.network import Network
import networkx as nx

nx_graph = nx.cycle_graph(10)
nx_graph.nodes[1]['title'] = 'Number 1'
nx_graph.nodes[1]['group'] = 1
nx_graph.nodes[3]['title'] = 'I belong to a different group!'
nx_graph.nodes[3]['group'] = 10
nx_graph.add_node(20, size=20, title='couple', label='2', group=2)
nx_graph.add_node(21, size=15, title='couple', group=2)

nx_graph.add_edge(20, 21, weight=5, title='7', label='5')
nx_graph.add_node(25, size=25, label='lonely', title='lonely node', group=3)

nt = Network("500px", "500px", notebook=True)
nt.from_nx(nx_graph)
# nt.show_buttons(filter_=['physics'])
nt.show_buttons()
nt.show("nx.html")


In [87]:
import dgl
import networkx as nx
from pyvis.network import Network

# Load the cora dataset and slice it using DGL
dataset = dgl.data.CoraGraphDataset()
g_dgl = dataset[0].subgraph(list(range(50)))

# Convert the DGL graph to NetworkX
g_netx = nx.Graph(g_dgl.to_networkx())

# Get the colors dict from labels (the class of the paper) 
labels = g_dgl.ndata['label'].numpy()
c_dict = {4:'red', 3:'black'}
colors = {i:c_dict.get(labels[i], 'blue') for i in range(50)}
# print("colors:", colors)

# Assign colors proportional to the labels found in DGL
sizes = {i:int(labels[i])+1 for i in range(50)}

# Add the colors and sizes to networkX graph as node attributes
nx.set_node_attributes(g_netx, colors, name="color")
nx.set_node_attributes(g_netx, sizes, name="size")
print(f"\nNode 0 color: {g_netx.nodes[0]['color']}")
print(f"Node 0 size: {g_netx.nodes[0]['size']}")

# Remap the node ids to strings to avoid error with PyVis
mapping = {i:str(i) for i in range(g_netx.size())}   # Setting mapping for the relabeling
g_netx = nx.relabel_nodes(g_netx, mapping)           # Relabeling nodes

# Plot the resulting netwrokx graph using PyVis
g_pyvis = Network(height=800, width=800, notebook=True)
g_pyvis.from_nx(g_netx, node_size_transf=lambda n:5*n)
g_pyvis.show('res.html')

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.

Node 0 color: red
Node 0 size: 5
